# Scraping Startup Data from NIC (National Incubation Center) Hyderabad Sindh, Pakistan

## Introduction

The National Incubation Center (NIC) Hyderabad is a hub for fostering innovation and entrepreneurship in Sindh, Pakistan. This notebook aims to scrape and compile data on startups from the NIC Hyderabad website. The objective is to gather information on the startups, including their names, descriptions, and links to their detailed profiles, which can be used for further analysis and research.

## 1. Setup and Libraries

In this section, we will import the necessary libraries required for web scraping and data processing.

In [1]:
import pandas as pd 
import requests
from bs4 import BeautifulSoup

## 3. Web Scraping
### Base URL and Headers
We define the base URL for NIC Hyderabad cohorts and set headers to mimic a real browser.

In [2]:
# Base URL adding 1, 2, 3, end of will make the change the round/cohort
url = 'https://nichyderabad.com/about/cohort-'

# Set headers to mimic a real browser
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}

## 3. Fetching Data from NIC Hyderabad
We fetch the web pages for each cohort and parse the HTML content.

In [3]:
# Initialize a dictionary to temporarily save all the responses
responses = {'cohorts': [], 'sope': []}

# Iterate over all the cohort pages (assuming there are 4 cohorts as of 24/06/2024)
for i in range(1, 5):  # till this date 24/06/2024 we only have four cohorts
    # Fetch the page of the current index
    response = requests.get(url + f'{i}', headers=headers)
    
    # Only if the request is successful
    if response.status_code == 200:
        # Save the cohort number to the responses
        responses['cohorts'].append(i)
        
        # Turn the response content into BeautifulSoup object
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Save the BeautifulSoup object to the responses
        responses['sope'].append(soup)
    else:
        # Print the status code if the request failed
        print(response.status_code, 'for searching cohort', i)


### Extracting Data for Each Cohort
We extract the relevant data (company names, URL of the each startup's profile page, and cohort session) from each cohort's page.
#### Cohort 1

In [4]:
# for cohort 1
soup_cohort1 = responses['sope'][0]

In [5]:
# selecting all "div" eiliments with class content-container
content_container = soup_cohort1.find_all('div', class_='content-container')
print('Total number of retrieve compony in cohort-1', len(content_container))

Total number of retrieve compony in cohort-1 20


In [6]:
# Initialize a dictionary to store NIC URLs and cohort numbers
NIC = {'NIC-URL': [], 'cohort': []}

# Iterate over all elements in content_container
for i in content_container:
    # Check if the element contains any <a> tags within a <p> tag
    if i.find('p') and i.find('p').find_all('a') != []:
        # Get the first <a> tag's href attribute (URL)
        startup_url = i.find('p').find_all('a')[0].get('href')
        # Append the URL to the 'NIC-URL' list in the NIC dictionary
        NIC["NIC-URL"].append(startup_url)

# Assign a cohort number of 1 to each NIC URL
NIC["cohort"] = [1] * len(NIC["NIC-URL"])


In [7]:
# changing it to Pandas Data Frame
NIC_df = pd.DataFrame(NIC)
NIC_df.head(20)

,NIC-URL,cohort
0,https://nichyderabad.com/about/cohort-1/emplai/,1
1,https://nichyderabad.com/about/cohort-1/agridu...,1
2,https://nichyderabad.com/about/cohort-1/seevit...,1
3,https://nichyderabad.com/about/cohort-1/monitr/,1
4,https://nichyderabad.com/about/cohort-1/upni-m...,1
5,https://nichyderabad.com/about/cohort-1/techfu...,1
6,https://nichyderabad.com/about/cohort-1/pak-bi...,1
7,https://nichyderabad.com/about/cohort-1/qualit...,1
8,https://nichyderabad.com/about/cohort-1/rentary/,1
9,https://nichyderabad.com/about/cohort-1/blocks...,1


#### Fetching the all the other cohorts

In [8]:
total_page = len(responses['sope'])
# cohort 2 and on
for i in range(1, total_page):
  sope_cohort = responses['sope'][i]
  # find all "a" tages with heading-link class ans store in list
  a_tags = sope_cohort.find_all('a', class_='heading-link')
  print(f'Total number of retrieve compony in cohort-{i+1} are', len(a_tags))
  for j in a_tags:
    # add the company, url, and cohort 2 to the NIC_df
    NIC_df.loc[len(NIC_df)] = [j.get('href'), 2]

Total number of retrieve compony in cohort-2 are 28
Total number of retrieve compony in cohort-3 are 20
Total number of retrieve compony in cohort-4 are 14


In [9]:
# Identify all rows where the 'NIC-URL' column has duplicate values
duplicates = NIC_df[NIC_df['NIC-URL'].duplicated(keep=False)]
duplicates

,NIC-URL,cohort
27,https://nichyderabad.com/about/cohort-2/digimart/,2
53,https://nichyderabad.com/about/cohort-2/digimart/,2


In [10]:
# Remove duplicate rows based on the 'NIC-URL' column
NIC_df.drop_duplicates(subset='NIC-URL', inplace=True)


*Note: Now that we have the like for each page of the startup on NIC website, we will start to fatch the data from thos page*

In [11]:
def get_soup(link_: str) -> BeautifulSoup:
    # Send an HTTP GET request to the specified link
    response = requests.get(link_, headers=headers)
    
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the response content using BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')
        # Return the BeautifulSoup object
        return soup
    else:
        # Print the status code if the request failed
        print(response.status_code, 'for', link_)

In [12]:
import re

# Iterate over the index of each URL in the 'NIC-URL' column of NIC_df
for i in NIC_df['NIC-URL'].index:
    # Fetch the HTML content of the page using the get_soup function
    soup = get_soup(NIC_df['NIC-URL'][i])
    
    # Find the <div> element that matches the specified class pattern
    table_div = soup.find('div', class_=re.compile(r'fusion-text fusion-text-2.*'))
    
    # Print the current index and the NIC URL
    print(i, NIC_df['NIC-URL'][i])
    
    # Find the <table> element within the located <div> element
    table = table_div.find('table')

    # Iterate over each row in the table
    for row in table.find_all('tr'):
        
        # Extract columns from each row
        columns = row.find_all('td')
        
        # Get the text content of each column and strip any extra whitespace
        columns = [col.get_text(strip=True) for col in columns]

        # add this info to the NIC_df where only have three columns [company	NIC-URL	cohort]
        if len(columns) >= 2:
            NIC_df.loc[i, columns[0]] = columns[1]

0 https://nichyderabad.com/about/cohort-1/emplai/
1 https://nichyderabad.com/about/cohort-1/agridunya-technologies/
2 https://nichyderabad.com/about/cohort-1/seevitals-solutions/
3 https://nichyderabad.com/about/cohort-1/monitr/
4 https://nichyderabad.com/about/cohort-1/upni-market/
5 https://nichyderabad.com/about/cohort-1/techfuturic/
6 https://nichyderabad.com/about/cohort-1/pak-bioenergy/
7 https://nichyderabad.com/about/cohort-1/quality-butler-services/
8 https://nichyderabad.com/about/cohort-1/rentary/
9 https://nichyderabad.com/about/cohort-1/blockshippro/
10 https://nichyderabad.com/about/cohort-1/salon-tracer/
11 https://nichyderabad.com/about/cohort-1/nanotechx/
12 https://nichyderabad.com/about/cohort-1/dbargain/
13 https://nichyderabad.com/about/cohort-1/sahulat/
14 https://nichyderabad.com/about/cohort-1/gems-rock-bid/
15 https://nichyderabad.com/about/cohort-1/verbue/
16 https://nichyderabad.com/about/cohort-1/mixeal/
17 https://nichyderabad.com/about/cohort-1/alt-ed/
18 

In [13]:
NIC_df.columns

Index(['NIC-URL', 'cohort', 'STARTUP', 'STARTUP STAGE', 'DOMAIN', 'WEBSITE',
       'FOUNDER', 'CONTACT', 'EMAIL'],
      dtype='object')

In [14]:
# Replace empty strings ('') and en dash ('–') with pd.NA (missing values)
NIC_df = NIC_df.replace('', pd.NA).replace('–', pd.NA)

In [15]:
# Remove all spaces from strings in the 'CONTACT' column
NIC_df['CONTACT'] = NIC_df['CONTACT'].apply(
    lambda x: x.replace(' ', '') if isinstance(x, str) else x)

# Replace the leading '0' with '+92' for strings in the 'CONTACT' column
NIC_df['CONTACT'] = NIC_df['CONTACT'].apply(
    lambda x: x.replace('0'+x[1:], '+92'+x[1:]) if (isinstance(x, str) and x[:1] == '0') else x)


In [16]:
NIC_df.head()

,NIC-URL,cohort,STARTUP,STARTUP STAGE,DOMAIN,WEBSITE,FOUNDER,CONTACT,EMAIL
0,https://nichyderabad.com/about/cohort-1/emplai/,1,EmplAi,Accelerate,"HR, Attendance",http://www.emplai.ai/,Aftab Ahmed Saraz (CEO & Founder),<NA>,<NA>
1,https://nichyderabad.com/about/cohort-1/agridu...,1,AgriDunya Technologies,Accelerate,AgTech,https://www.agridunya.com/,Rahul Dembani,+923300792000,support@agridunya.com
2,https://nichyderabad.com/about/cohort-1/seevit...,1,SeeVitals,Accelerate,Health Tech,http://www.seevitals.com/,Dr. Nimra Qureshi,+923451591251,info@SeeVitals.com
3,https://nichyderabad.com/about/cohort-1/monitr/,1,Monitr,Accelerate,"Ed-Tech, SAAS",www.monitr.site,Daeyan Hafeez Siddiqui,+923453531573,daeyansidi826@gmail.com
4,https://nichyderabad.com/about/cohort-1/upni-m...,1,Upni Market,Accelerate,E-commerce,www.Upnimarket.com,Ahsan Zahid,+923220232991,info@upnimarket.com


## 4. Saving the Data
Finally, we save the collected and processed data to a CSV file for future analysis.

In [17]:
NIC_df.to_csv('NIC-hydrabad_startups-data.csv')

## Accessing the Data

The dataset generated from this notebook can be accessed through GitHub: [NIC Startups Data](https://github.com/codewithawr/NIC-hydrabad-Startups-Data)

## Appendix
### Additional Information
- Source: National Incubation Center (NIC) Hyderabad website
- Date of Data Collection: 24/06/2024
- Acknowledgments: Thanks to NIC Hyderabad for providing access to the startup information